### DATA CLEANING - 2

Please read data_cleaning_updated for details about the initial steps for data analysis.
After running the models and investigatig further the data the following decisions were taken:

<b> Feature extraction: </b>

- Remove outliers by calculating 6 * IQR. The reason behind this is that normally an outlier is consider as something that is above 1.5 IQR. In this case because we have ill patients we consider bigger IQR as plausible values.
- Create a feature BMI and remove from the dataframe Weight and Heigh
- For Mechanical ventilation assume Nan as no mechanical ventilation (assign this values to 0)
- For every time parameter extract:
   - median
   - median at 24 hours
   - median at 48 hours
   - normalized MAD
   - max
   - last value
   - slope

<b> Feature selection: </b> 
- Looking for features that are not representative and with excessive NaN values: Eliminate features were more than 50% of participants have missing values

- Look at correlations amongst features and also correlations amongst features and eliminate those features with a correlation > 0.8 

- Drop low variance features:  Zero or near-zero variance. Features that are (almost) constant provide little information to learn from and thus are irrelevant.
   

In [198]:
import pandas as pd
import pandas as pd
import glob
from pathlib import Path  # Importing the Path class from the pathlib module
import numpy as np
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os 
import math
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import RobustScaler

In [199]:
#GLOBAL VARIABLES
# Define the programmer whose path you want to use
programmer = 'esther'

#variables for paths names
file_paths_names = {
    'patricia': r"..\original_data",
    'esther': r"C:\Users\egh22\OneDrive - University of Canterbury\bootcamp\group project\data"
}

files_outcome_path_names = {
    'patricia': r"..\original_data\outcomes",
    'esther': r"C:\Users\egh22\OneDrive - University of Canterbury\bootcamp\group project\data\outcomes"
}


files_path_to_save_files = {
    'patricia': r"..\clean_data",
    'esther': r"C:\Users\egh22\OneDrive - University of Canterbury\bootcamp\group project\clean_data"
}

# Choose the file path based on the programmer variable
file_path_to_save = files_path_to_save_files[programmer]


# VARIABLES FOR FEATURE REDUCTION
#threshold variables
#defines the number of std needed for a value to be considered out of range
threshold_for_outliers = 6
#defines the percentage of patients with NaNs for a feature  to be deleted from dataframe
thresholds_for_non_important_features = 50
#variance reduction threshold 
thresholds_variance_reduction = 0.05

In [200]:
def timeit(func):
    """
    Decorator function to measure the execution time of another function.

    Parameters:
        func (callable): The function to be decorated.

    Returns:
        callable: A wrapper function that measures the execution time of the decorated function.
    """
    def wrapper(*args, **kwargs):
        """
        Wrapper function that measures the execution time of the decorated function.

        Parameters:
            *args: Positional arguments to be passed to the decorated function.
            **kwargs: Keyword arguments to be passed to the decorated function.

        Returns:
            Any: The return value of the decorated function.
        """
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' elapsed time: {elapsed_time} seconds")
        return result
    return wrapper


### 1. Feature extraction

For every time parameter extract:
   - median
   - median at 24 hours
   - median at 48 hours
   - normalized MAD
   - max
   - last value
   - slope


In [201]:
@timeit
def process_time_series_files(time_series_files: list, all_parameters: list, non_time_parameters: list, categorical_parameters:list, summary_function: callable) -> pd.DataFrame:
    """
    Process a list of time series files and extract summary statistics for each file using a specified summary function.

    Parameters:
        time_series_files (list): A list of file paths to time series data files.
        all_parameters (list): A list of all parameters to calculate metrics for.
        non_time_parameters (list): A list of non-time parameters to extract values for.
        categorical_parameters (list): A list of special non-time parameters, that may appear more than once over time but always takes same value to extract values for.
        summary_function (callable): A function to calculate summary statistics for a DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the summary statistics for all files.
    """
    results = []

    for file_path in time_series_files:
        # Read the file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract summary statistics for the DataFrame using the specified summary function
        result = summary_function(df, all_parameters, non_time_parameters, categorical_parameters)
        
        # Append the result to the results list
        results.append(result)

    # Concatenate results into a single DataFrame
    final_result = pd.concat(results, ignore_index=True)
    
    return final_result


In [202]:
# make a list with all time parameters in the dataset
time_parameters =  ['Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 
                    'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K',
                    'Lactate', 'Mg', 'MAP', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 
                 'PaO2',  'pH', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TropI', 'TropT', 'Urine', 'WBC',  'Weight']

non_time_parameters = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType']
demographic_parameters = non_time_parameters

categorical_parameters = ['MechVent']

# Concatenate all the lists
all_parameters = time_parameters + non_time_parameters + categorical_parameters

In [203]:
# Specify the file path using a raw string
#Modify path to your desired one
files_path = file_paths_names[programmer]

# Use glob to find all .txt files in the directory
time_series_files = glob.glob(str(Path(files_path) / "*.txt"))  # Using Path to construct the file path

# Print the number of files found
print(f"Number of text files: {len(time_series_files)}")
#delete this after demonstration
time_series_files_test = time_series_files[1]
#df = pd.read_csv(time_series_files[1])
time_series_files_test

Number of text files: 8000


'C:\\Users\\egh22\\OneDrive - University of Canterbury\\bootcamp\\group project\\data\\132540.txt'

In [204]:
# read target variables

#modify this with the name of the paths to read
files_outcome_path =  files_outcome_path_names[programmer]
df_target_a = pd.read_csv(Path(files_outcome_path) / "Outcomes-a.txt")
df_target_b = pd.read_csv(Path(files_outcome_path) / "Outcomes-b.txt")

outcomes_df = pd.concat([df_target_a, df_target_b], ignore_index=True)
outcome_columns = outcomes_df.columns.tolist()
outcome_columns.remove('RecordID')

In [205]:
def extract_summarysed_features(df, time_parameters_filtered, demographic_parameters, categorical_parameters):
    """
    Generate new time summarised features based on different statistics and extract the values for demographic parameters, categorical parameters, 
    from the given DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing time-series data.
        time_parameters_filtered (list): A list of time-related parameters to extract statistics for.
        demographic_parameters (list): A list of demographic parameters to extract.
        categorical_parameters (list): A list of categorical parameters to extract.
    
    Returns:
        pd.DataFrame: A DataFrame containing the extracted values for demographic parameters,
                      categorical parameters, and various statistics for time parameters.
    """
    # Replace -1.0 with NaN
    df = df.replace(-1.0, np.nan)
    
    # Convert "Time" column to string
    df['Time'] = df['Time'].astype(str)
    
    # Extract hour from "Time" column
    df['Hour'] = df['Time'].str.split(':').str[0].astype(int)
    
    # Pivot the dataframe to have features as columns
    df_pivot = df.pivot_table(index='Hour', columns='Parameter', values='Value', aggfunc='median')
    
    # Create a new dataframe to store the extracted values
    new_df = pd.DataFrame()
    
    # Extract values for demographic parameters
    demographic_data = {}
    for feature in demographic_parameters:
        if feature in df_pivot.columns:
            demographic_data[feature] = df_pivot[feature].iloc[0]  # Extract the first value
        else:
            demographic_data[feature] = np.nan  # Feature not found, assign NaN
    demographic_df = pd.DataFrame(demographic_data, index=[0])

    # Extract values for categorical parameters
    categorical_data = {}
    for feature in categorical_parameters:
        if feature in df_pivot.columns:
            categorical_data[f'{feature}_start'] = df_pivot[df_pivot[feature].notna()].index.min()  # Extract the first value
            categorical_data[feature] = df_pivot.loc[df_pivot[df_pivot[feature].notna()].index.min(), feature]
        else:
            categorical_data[feature] = np.nan  # Feature not found, assign NaN
            categorical_data[f'{feature}_start'] = np.nan  # Feature not found, assign NaN
    categorical_df = pd.DataFrame(categorical_data, index=[0])

    
    # Extract median from 0-24, median from 25-48, overal_median, mad, max, slope and last_available value
    # for all time parameters
    time_data = {}
    
    for feature in time_parameters_filtered:
        if feature in df_pivot.columns:
            # Remove outliers for the slope calculation
            Q1 = df_pivot[feature].quantile(0.25)
            Q3 = df_pivot[feature].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold_for_outliers * IQR
            upper_bound = Q3 + threshold_for_outliers * IQR
            df_pivot[feature] = df_pivot[feature].where((df_pivot[feature] >= lower_bound) & (df_pivot[feature] <= upper_bound))
             
            # Calculate median value for 0-24 hours
            time_data[f'{feature}_24median'] = df_pivot.loc[0:24, feature].median()
            # Calculate median value for 25-48 hours
            time_data[f'{feature}_48median'] = df_pivot.loc[25:48, feature].median()
            # Calculate overall median value
            time_data[f'{feature}_median'] = df_pivot[feature].median()
            # Calculate max value
            time_data[f'{feature}_max'] = df_pivot[feature].max()
            #calculate mad
            # Calculate mean absolute deviation (MAD) normalised
            # Calculate the absolute differences between each value and the median
            absolute_diff_median = (np.abs(df_pivot[feature] - df_pivot[feature].median())).median()
            # Calculate the median of these absolute differences
            time_data[f'{feature}_mad'] = ((absolute_diff_median/df_pivot[feature].median())*100) if df_pivot[feature].median() != 0 else np.nan
            # Calculate last available value
            time_data[f'{feature}_last'] = df_pivot[feature].dropna().iloc[-1] if not df_pivot[feature].isnull().all() else np.nan
            # Fit linear regression to the data
            # Flatten the data for the feature across all hours
            feature_values = df_pivot[feature].dropna().values
            all_hours = df_pivot[feature].dropna().index.values
            
            if len(all_hours) > 1:
                # Fit a linear regression line to the data
                coefficients = np.polyfit(all_hours, feature_values, 1)
                # Store the slope in the dataframe
                time_data[f'{feature}_slope'] = coefficients[0]
            else:
                # Insufficient data points for linear regression, assign NaN
                time_data[f'{feature}_slope'] = np.nan
           
        else:
             # Calculate median value for 0-24 hours
            time_data[f'{feature}_24median'] = np.nan
            time_data[f'{feature}_48median'] = np.nan
            time_data[f'{feature}_median'] = np.nan
            time_data[f'{feature}_max'] = np.nan
            time_data[f'{feature}_mad'] = np.nan
            time_data[f'{feature}_last'] = np.nan
            time_data[f'{feature}_slope'] = np.nan

    time_df = pd.DataFrame(time_data, index=[0])
 
    
    # Concatenate all dataframes along columns axis
    final_df = pd.concat([demographic_df, categorical_df, time_df], axis=1)
    
    return final_df

#Uncomment this to create file!!
#save dataframe for feature cleaning
#result = process_time_series_files(time_series_files, time_parameters, demographic_parameters, categorical_parameters, extract_summarysed_features)
#result.to_csv(os.path.join(file_path_to_save, "uncleaned_summarysed_features.csv"), index=False)

In [206]:
final_dataframe = pd.read_csv(os.path.join(file_path_to_save, 'uncleaned_summarysed_features.csv'))
print("Number of columns after feature extraction:", final_dataframe.shape[1])

Number of columns after feature extraction: 259


In [207]:
#impute Mech_vent Nan values with 0
final_dataframe['MechVent'] = final_dataframe['MechVent'].fillna(0)

#Create BMI value and drop 'Height', 'Weight_median'
final_dataframe['BMI'] = np.where(final_dataframe['Height'].isnull() | final_dataframe['Weight_median'].isnull(), np.nan, (final_dataframe['Weight_median'] / ((final_dataframe['Height'] / 100) ** 2)))
                               
#Add BMI to my demo features
demographic_parameters.append('BMI')
demographic_parameters.remove('Height')
#Reorder dataframe so that demo parameters go first 

# Get a list of all column names
columns = list(final_dataframe.columns)

# Move demographic parameters to the beginning of the list
for param in demographic_parameters[::-1]:
    columns.remove(param)
    columns.insert(0, param)

# Reorder the DataFrame columns
final_dataframe = final_dataframe[columns]
final_dataframe.drop(columns=['Height', 'Weight_median'], inplace=True)   
final_dataframe.head(5)

,RecordID,Age,Gender,ICUType,BMI,MechVent,MechVent_start,Albumin_24median,Albumin_48median,Albumin_median,...,WBC_max,WBC_mad,WBC_last,WBC_slope,Weight_24median,Weight_48median,Weight_max,Weight_mad,Weight_last,Weight_slope
0,132539.0,54.0,0.0,4.0,NaN,0.0,NaN,NaN,NaN,NaN,...,11.2,8.737864,9.4,-0.078261,NaN,NaN,NaN,NaN,NaN,NaN
1,132540.0,76.0,1.0,2.0,26.228364,1.0,1.0,NaN,NaN,NaN,...,13.3,1.526718,13.3,0.126301,80.6,80.6,80.6,0.0,80.6,5.728184e-16
2,132541.0,44.0,0.0,3.0,NaN,1.0,10.0,2.5,NaN,2.5,...,6.2,11.904762,6.2,0.058981,56.7,56.7,56.7,0.0,56.7,-7.200356e-16
3,132543.0,68.0,1.0,3.0,26.024291,0.0,NaN,4.4,NaN,4.4,...,11.5,10.227273,7.9,-0.095192,84.6,84.6,84.6,0.0,84.6,-5.533244e-16
4,132545.0,88.0,0.0,3.0,NaN,0.0,NaN,NaN,3.3,3.3,...,4.8,11.627907,4.8,0.038462,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Feature Selection

##### Removing features with more than 50% NaN values

In [208]:
# Calculate percentage of missing values per feature
total_rows = len(final_dataframe)
missing_values_percentage = (final_dataframe.isnull().sum() / total_rows) * 100

# Create a DataFrame to store missing values percentage
missing_percentage_table = pd.DataFrame({
    'Feature': missing_values_percentage.index,
    'Missing Percentage': missing_values_percentage.values
})

# Sort missing percentage table by Missing Percentage column in descending order
missing_percentage_table_sorted = missing_percentage_table.sort_values(by='Missing Percentage', ascending=False)

# list features with missing values >50%
features_missing_gt_50 = missing_percentage_table_sorted[missing_percentage_table_sorted['Missing Percentage'] >= thresholds_for_non_important_features]
print("Number of features missing more than 50% of values:", features_missing_gt_50.shape[0])
# List the feature names
print("Name fo the features missing more than 50% of values:", features_missing_gt_50['Feature'].tolist())
features_missing_gt_50

Number of features missing more than 50% of values: 73
Name fo the features missing more than 50% of values: ['TropI_max', 'TropI_mad', 'TropI_last', 'TropI_slope', 'TropT_24median', 'TropT_48median', 'TropI_median', 'TropI_48median', 'TropI_24median', 'TropT_median', 'TropT_max', 'TropT_mad', 'TropT_last', 'TropT_slope', 'Cholesterol_slope', 'Cholesterol_48median', 'Cholesterol_24median', 'Cholesterol_max', 'Cholesterol_median', 'Cholesterol_last', 'Cholesterol_mad', 'Albumin_slope', 'Albumin_48median', 'ALP_slope', 'AST_slope', 'ALT_slope', 'Bilirubin_slope', 'ALP_48median', 'AST_48median', 'ALT_48median', 'Bilirubin_48median', 'Lactate_48median', 'SaO2_48median', 'RespRate_48median', 'RespRate_24median', 'RespRate_slope', 'RespRate_last', 'RespRate_mad', 'RespRate_median', 'RespRate_max', 'SaO2_slope', 'Albumin_24median', 'Lactate_slope', 'ALP_24median', 'Bilirubin_24median', 'SaO2_24median', 'ALT_24median', 'AST_24median', 'Albumin_mad', 'Albumin_median', 'Albumin_max', 'Albumin_la

,Feature,Missing Percentage
227,TropI_max,100.0000
228,TropI_mad,100.0000
229,TropI_last,100.0000
230,TropI_slope,100.0000
231,TropT_24median,100.0000
...,...,...
207,SaO2_mad,55.6875
208,SaO2_last,55.6875
206,SaO2_max,55.6875
205,SaO2_median,55.6875


In [209]:
# Create a boolean mask for features with missing percentage >= 70
mask = missing_values_percentage >= thresholds_for_non_important_features 
final_dataframe_filtered = final_dataframe.loc[:, ~mask]
print("Number of features after eliminating >50% missing features:", final_dataframe_filtered.shape[1])

Number of features after eliminating >50% missing features: 185


#### Removing low variance features

In [210]:
X = final_dataframe_filtered.drop(columns=demographic_parameters + categorical_parameters)

# Step 1: Scale your features using RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Instantiate the VarianceThreshold object with the desired threshold
# Step 2: Use VarianceThreshold to select features based on variance
threshold = 0.05  # Set your desired threshold
sel = VarianceThreshold(threshold=threshold)
X_selection = sel.fit_transform(X_scaled)

# Get boolean mask of selected features
selected_columns_mask = sel.get_support()

# Get selected column names
selected_columns = X.columns[selected_columns_mask]

# Get dropped column names
dropped_columns = X.columns[~selected_columns_mask]

# Print selected and dropped column names
print("Selected columns:", selected_columns)
print("Dropped columns:", dropped_columns)

Selected columns: Index(['MechVent_start', 'BUN_24median', 'BUN_48median', 'BUN_median',
       'BUN_max', 'BUN_mad', 'BUN_last', 'BUN_slope', 'Creatinine_24median',
       'Creatinine_48median',
       ...
       'WBC_median', 'WBC_max', 'WBC_mad', 'WBC_last', 'WBC_slope',
       'Weight_24median', 'Weight_48median', 'Weight_max', 'Weight_mad',
       'Weight_last'],
      dtype='object', length=178)
Dropped columns: Index(['Weight_slope'], dtype='object')


In [211]:
final_dataframe_filtered = final_dataframe_filtered.drop(columns=dropped_columns.tolist())
print("Number of columns after low variance dropped:", final_dataframe_filtered.shape[1])

Number of columns after low variance dropped: 184


#### Removing high correlated features

In [212]:
dataframe_4correlation = final_dataframe_filtered.drop(columns=demographic_parameters + categorical_parameters)

#Re-order the dataframe so that we prioritize keeping the median features just because it is easy to interprete later
median_cols = [col for col in dataframe_4correlation.columns if col.endswith("_median")]
non_median_cols = [col for col in dataframe_4correlation.columns if not col.endswith("_median")]
ordered_cols = median_cols + non_median_cols

# Create the reordered DataFrame
dataframe_4correlation_reordered = dataframe_4correlation[ordered_cols]


In [213]:
# Calculate correlation matrix

corr_matrix = dataframe_4correlation_reordered.corr().abs()
# Set upper triangle of correlation matrix to NaN
corr_matrix.values[np.triu_indices_from(corr_matrix)] = np.nan

# Find highly correlated features
highly_correlated_features = set()
for col in corr_matrix.columns:
    correlated_cols = corr_matrix[col][corr_matrix[col] > 0.8].index.tolist()
    #print(col, ",", correlated_cols)
    # Drop the first column in each pair of highly correlated features
    highly_correlated_features.update(correlated_cols)

final_dataframe = final_dataframe_filtered.drop(columns=highly_correlated_features)


In [214]:
# oduble checking we haven't missed dropping any columns
corr_matrix = final_dataframe.corr().abs()
threshold = 0.8  # Choose a threshold

# find correlations
high_correlations = find_high_correlations(corr_matrix, threshold)

# Convert the list to a DataFrame for easier manipulation
high_correlations_df = pd.DataFrame(high_correlations, columns=['Feature 1', 'Feature 2', 'Correlation Coefficient'])

# Display the DataFrame
high_correlations_df.to_csv(os.path.join(file_path_to_save, "high_correlated_variables.csv"), index=False)
high_correlations_df

,Feature 1,Feature 2,Correlation Coefficient


In [216]:
print("Number of columns after correlation features dropped:", final_dataframe.shape[1])

Number of columns after correlation features dropped: 99


In [217]:
#save dataframe 
final_dataframe.to_csv(os.path.join(file_path_to_save, "clean_dataframe_with_NaN.csv"), index=False)

Now we have our final df saved and ready for next step : NaN imputation